## Advise re identity architecture

Initialize

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.0.1-alpha"

#r "nuget: System.Linq.Async, 6.0.1"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;

using Kernel = Microsoft.SemanticKernel.Kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, bingApiKey, orgId) = Settings.LoadFromFile();
var kernel = Microsoft.SemanticKernel.Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion(
                model,   // deployment name
                azureEndpoint, // Azure OpenAI Endpoint
                apiKey)      // Azure OpenAI Key
            .Build();
#pragma warning disable SKEXP0011, SKEXP0003, SKEXP0052
var memoryBuilder = new MemoryBuilder();
memoryBuilder.WithAzureOpenAITextEmbeddingGeneration(
    "mrtextembeddingada002",
    azureEndpoint, 
    apiKey)
.WithMemoryStore(new VolatileMemoryStore());   
var memory = memoryBuilder.Build();            

Installed Packages Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Plugins.Memory, 1.0.1-alpha System.Linq.Async, 6.0.1

Define/load plugin to facilitate use of chat history

In [2]:
using Microsoft.SemanticKernel.Plugins.Memory;

kernel.Plugins.Clear();

#pragma warning disable SKEXP0052

// TextMemoryPlugin provides the "recall" function
kernel.ImportPluginFromObject(new TextMemoryPlugin(memory));
var appFunction = kernel.CreateFunctionFromPrompt(File.ReadAllText("./ApplicationDiscovery.txt"), new OpenAIPromptExecutionSettings { MaxTokens = 200, Temperature = 0.8 });
var userFunction = kernel.CreateFunctionFromPrompt(File.ReadAllText("./UserIdentity.txt"), new OpenAIPromptExecutionSettings { MaxTokens = 200, Temperature = 0.0 });

Define arguments

In [3]:
#pragma warning disable SKEXP0052
var arguments = new KernelArguments();
arguments[TextMemoryPlugin.CollectionParam] = "history";
arguments[TextMemoryPlugin.LimitParam] = "2";
arguments[TextMemoryPlugin.RelevanceParam] = "0.8";

Define chat item

In [4]:
var history = "";
arguments["history"] = history;
Func<string, Task> Chat = async (string input) => {
    arguments["userInput"] = input;
    var answer = await userFunction.InvokeAsync(kernel, arguments);
    var result = $"\nUser: {input}\nassistant: {answer}\n";
    history += result;
    arguments["history"] = history;
    Console.WriteLine(result);
};

Define chat loop

In [5]:
using System.Globalization;
using Microsoft.DotNet.Interactive;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;
do
{
    var user = await InteractiveKernel.GetInputAsync("Please enter your question or response");
    if (String.IsNullOrEmpty(user) || user.StartsWith("q", true, CultureInfo.CurrentCulture)) break;
    await Chat(user);
} while (true);
Console.WriteLine("Finished.");


User: We have a desktop app for our employees
assistant: Based on the information provided, I would recommend using Entra ID as the token issuer for your application. Entra ID allows you to control the authorization of your employees and decide who can use the application. This option would be suitable if you have full control over the authorization process and do not require integration with external directories or consumer users.


User: But it will also be used by our partners
assistant: In that case, I would recommend using Entra ID, optionally with B2B. This option allows you to control the authorization of your employees and partners, giving you full control over who can access the application. With Entra ID B2B, you can easily invite and manage external users from partner organizations. This option would be suitable if you want to have control over the authorization process for both employees and partners, without relying on each company to manage their own authorization.

Fini

Some tests

In [13]:
await Chat("Here is my system description: it consists of a single web application showing html content.");


User: Here is my system description: it consists of a single web application showing html content.
ChatBot: Got it. So your system consists of a single web application that shows HTML content. Is that correct?

